In [ ]:
from tqdm import tqdm
import json
import requests
import geopandas as gpd
import geocoder
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point, Polygon
from func import *

In [ ]:
specified_order = [
    '北投區', '士林區', '內湖區', '南港區',
    '松山區', '信義區', '中山區', '大同區',
    '中正區', '萬華區', '大安區', '文山區'
]